In [ ]:
import os
# import pickle

import cv2
import numpy as np
from PIL import Image
import albumentations as A

In [ ]:
# Ruta al directorio donde se encuentran las carpetas de imágenes
data_dir = 'D:\\Descargas\\IDC_regular_ps50_idx5'

# Ruta de salida dentro del proyecto
output_dir = 'D:\\Descargas\\IDC_regular_ps50_idx5_v2'

# Tamaño de cada imagen existente
image_size = 50

# Definimos las dimensiones a las que redimensionaremos las imágenes generadas
IMG_WIDTH, IMG_HEIGHT = 50, 50

# Listas para almacenar las imágenes y las etiquetas
all_images = []
all_labels = []

# Definimos el data augmentation
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=90, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    # A.ElasticTransform(p=0.5),
])

num_augmented_images = 9  # Número de imágenes aumentadas a generar por cada imagen original

In [ ]:
def generate_class_image(patient_id, class_id):
    images = []
    coordinates = []

    image_files = os.listdir(os.path.join(data_dir, patient_id, class_id))
    for image_file in image_files:
        # Extraemos las coordenadas x e y de los nombres de archivo
        filename = os.path.splitext(image_file)[0]
        parts = filename.split('_')
        x = int(parts[-3][1:])
        y = int(parts[-2][1:])
        coordinates.append((x, y))

        # Cargamos la imagen
        image_path = os.path.join(data_dir, patient_id, class_id, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (image_size, image_size))  # Redimensionamos a 50x50
        images.append(image)

    # Ordenamos las coordenadas en orden ascendente de proximidad
    coordinates, images = zip(*sorted(zip(coordinates, images), key=lambda x: sum(x[0])))

    # Calculamos las dimensiones de la imagen final
    max_x = max(coord[0] for coord in coordinates)
    max_y = max(coord[1] for coord in coordinates)
    final_width = max_x + image_size
    final_height = max_y + image_size
    final_shape = (final_height, final_width, 3)

    # Creamos una imagen final en blanco con las dimensiones adecuadas
    final_image = np.zeros(final_shape, dtype=np.uint8)

    # Colocamos las imágenes en la posición correcta en la imagen final
    for coord, image in zip(coordinates, images):
        x, y = coord
        final_image[y:y + image_size, x:x + image_size] = image

    # Redimensionamos la imagen a 768x768
    final_image = cv2.resize(final_image, (IMG_WIDTH, IMG_HEIGHT))

    # Guardamos la imagen final "normal" en la ruta del proyecto
    output_path = os.path.join(output_dir, patient_id, class_id, f"{patient_id}_idx5_class{class_id}.png")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    Image.fromarray(final_image).save(output_path)

    all_images.append(final_image)
    all_labels.append(int(class_id))

    print(f"Imagen normal generada para el paciente {patient_id}, clase {class_id}. La imagen final se ha guardado en:", output_path)

    # Aumentamos la imagen
    for i in range(num_augmented_images):
        augmented_final_image = aug(image=final_image)['image']

        # Redimensionamos la imagen a 768x768
        augmented_final_image = cv2.resize(augmented_final_image, (IMG_WIDTH, IMG_HEIGHT))

        # Guardamos la imagen final aumentada en la ruta del proyecto
        output_path = os.path.join(output_dir, patient_id, class_id, f"{patient_id}_idx5_class{class_id}_aug_{i}.png")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        Image.fromarray(augmented_final_image).save(output_path)

        all_images.append(augmented_final_image)
        all_labels.append(int(class_id))

        print(f"Imagen aumentada #{i} generada para el paciente {patient_id}, clase {class_id}. La imagen final se ha guardado en:", output_path)

In [ ]:
# Iteramos sobre todas las carpetas de pacientes
for patient_id in os.listdir(data_dir):
    if not patient_id.startswith('.'):  # Ignoramos archivos/carpetas como .DS_Store
        print(f"Procesando el paciente {patient_id}...")

        # Generar imágenes para la clase 0
        generate_class_image(patient_id, '0')

        # Generar imágenes para la clase 1
        generate_class_image(patient_id, '1')

        # Generar imagen general con ambas clases
        images = []
        coordinates = []

        for class_id in ['0', '1']:
            if not class_id.startswith('.'):  # Nuevamente, ignoramos archivos/carpetas como .DS_Store
                image_files = os.listdir(os.path.join(data_dir, patient_id, class_id))
                for image_file in image_files:
                    # Extraemos las coordenadas x e y de los nombres de archivo
                    filename = os.path.splitext(image_file)[0]
                    parts = filename.split('_')
                    x = int(parts[-3][1:])
                    y = int(parts[-2][1:])
                    coordinates.append((x, y))

                    # Cargamos y redimensionamos la imagen
                    image_path = os.path.join(data_dir, patient_id, class_id, image_file)
                    image = cv2.imread(image_path)
                    image = cv2.resize(image, (image_size, image_size))  # Redimensionamos a 50x50
                    images.append(image)

        # Ordenamos las coordenadas en orden ascendente de proximidad
        coordinates, images = zip(*sorted(zip(coordinates, images), key=lambda x: sum(x[0])))

        # Calculamos las dimensiones de la imagen final
        max_x = max(coord[0] for coord in coordinates)
        max_y = max(coord[1] for coord in coordinates)
        final_width = max_x + image_size
        final_height = max_y + image_size
        final_shape = (final_height, final_width, 3)

        # Creamos una imagen final en blanco con las dimensiones adecuadas
        final_image = np.zeros(final_shape, dtype=np.uint8)

        # Colocamos las imágenes en la posición correcta en la imagen final
        for coord, image in zip(coordinates, images):
            x, y = coord
            final_image[y:y + image_size, x:x + image_size] = image

        # Redimensionamos la imagen a 768x768
        final_image = cv2.resize(final_image, (IMG_WIDTH, IMG_HEIGHT))

        # Guardamos la imagen final en la ruta del proyecto
        output_path_general = os.path.join(output_dir, patient_id, f"{patient_id}_idx5.png")
        os.makedirs(os.path.dirname(output_path_general), exist_ok=True)
        Image.fromarray(final_image).save(output_path_general)

        print(f"Proceso completado para el paciente {patient_id}. La imagen final se ha guardado en:",
              output_path_general)

In [ ]:
# Convertimos las listas a arrays de numpy para que sean más eficientes
# images = np.array(all_images)
# labels = np.array(all_labels)

# Crear un diccionario con los nombres de las columnas
data = {'image': all_images, 'label': all_labels, 'area': 'breast'}

# Crear dataframe desde el diccionario
breast_df = pd.DataFrame(data, columns=['image', 'label', 'area'])

# Guardar el diccionario en un archivo pickle
output_file = 'breast_df.pkl'
breast_df.to_pickle(output_file)
print(f"Archivo pickle generado: {output_file}")

# with open(output_file, 'wb') as f:
#     pickle.dump(data, f)